In [33]:
import pandas as pd
import numpy as np

# Section 7: Going Multidimensional

In this section, we'll kick things up a notch and work with multi-index dataframes. This allows us to support more than one level of labels, enabling us to reflect multidimensional datasets within the confines of a two-dimensional data structure.

**Hierarchichal indices** represent a hierarchy of relationships that become intricately coupled with our data/values. Using Pandas techniques and methods, we can very efficiently change these hierarchies in order to answer specific questions.

As a side note, multi-level index, multiindex, and hierarchical index are all used interchangeably within the Pandas community.

## Introducing New Data

In this section, we'll be working with a new dataset that contains daily stock information from 2014 through 2019 for the technology companies Apple, Facebook, Microsoft, Google, and Amazon.

In [34]:
tech_url = 'https://andybek.com/pandas-tech'

In [35]:
tech = pd.read_csv(tech_url)

In [36]:
tech.head()

,date,month,year,day,name,open,close,high,low,volume,volume_type
0,2014-01-02,1,2014,2,FB,54.86,54.71,55.22,54.19,43257622,medium
1,2014-01-02,1,2014,2,AAPL,79.38,79.02,79.58,78.86,8398851,low
2,2014-01-02,1,2014,2,GOOGL,557.73,556.56,558.88,554.13,1822719,medium
3,2014-01-02,1,2014,2,MSFT,37.35,37.16,37.40,37.10,30643745,medium
4,2014-01-02,1,2014,2,AMZN,398.80,397.97,399.36,394.02,2140246,medium


Here we see we have date in the first column, then columns of decomposed dates, the stock tickers under "name", the opening, closing, high, and low prices for that day, the volume of shares traded, and the qualitative trade volume.

Examining the shape of the dataframe, we have 7105 rows and 11 columns.

In [37]:
tech.shape

(7105, 11)

Let's also check out the type of data in the dataframe.

In [38]:
tech.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7105 entries, 0 to 7104
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         7105 non-null   object 
 1   month        7105 non-null   int64  
 2   year         7105 non-null   int64  
 3   day          7105 non-null   int64  
 4   name         7105 non-null   object 
 5   open         7105 non-null   float64
 6   close        7105 non-null   float64
 7   high         7105 non-null   float64
 8   low          7105 non-null   float64
 9   volume       7105 non-null   int64  
 10  volume_type  7105 non-null   object 
dtypes: float64(4), int64(4), object(3)
memory usage: 610.7+ KB


We have 4 floating point columns (open, close, high, low), 4 integer columns (month, year, day, volume), and three object columns (date, name, and volume_type).

## Index and RangeIndex

Let's review the terminology that we've grown accustomed to. In Series, the index is simply a label for each value in the series. In dataframes, the index still serves as a label for each row, but is accompanied by another set of labels for the column dimension.

Remember that an index of the type `RangeIndex`, which is simply an immutable or unchangable object that represents a series of increasing or decreasing integers. In Pandas, `RangeIndex` is further derived from the `index` class, which happens to be the same exact class type that the column labels inherit from

In [39]:
type(tech.index)

pandas.core.indexes.range.RangeIndex

In [40]:
type(tech.columns)

pandas.core.indexes.base.Index

So what is the "index" class? It's essentially another immutable data structure (in this case a Numpy array) that is ordered and sliceable.

Oftentimes we find the need to replace the default indices with something more meaningful. We've previously seen this with the `set_index()` method.

In [41]:
tech.set_index('date')

,month,year,day,name,open,close,high,low,volume,volume_type
date,,,,,,,,,,
2014-01-02,1,2014,2,FB,54.86,54.71,55.22,54.19,43257622,medium
2014-01-02,1,2014,2,AAPL,79.38,79.02,79.58,78.86,8398851,low
2014-01-02,1,2014,2,GOOGL,557.73,556.56,558.88,554.13,1822719,medium
2014-01-02,1,2014,2,MSFT,37.35,37.16,37.40,37.10,30643745,medium
2014-01-02,1,2014,2,AMZN,398.80,397.97,399.36,394.02,2140246,medium
...,...,...,...,...,...,...,...,...,...,...
2019-08-23,8,2019,23,MSFT,137.19,133.39,138.35,132.80,38515386,medium
2019-08-23,8,2019,23,AAPL,209.43,202.64,212.05,201.00,46882843,medium
2019-08-23,8,2019,23,GOOGL,1185.17,1153.58,1195.67,1150.00,1813141,medium


But what does it mean for *date* to "be" an index? One of the key implications has to do with how we select from the dataframe. If we wanted to extract or index all the prices for August 1, 2019, we can use that for our selection.

In [42]:
tech.set_index('date').loc['2019-08-01']

,month,year,day,name,open,close,high,low,volume,volume_type
date,,,,,,,,,,
2019-08-01,8,2019,1,GOOGL,1217.63,1211.78,1236.30,1207.00,1771271,medium
2019-08-01,8,2019,1,FB,194.17,192.73,198.47,190.88,17777013,medium
2019-08-01,8,2019,1,MSFT,137.00,138.06,140.94,136.93,40557502,medium
2019-08-01,8,2019,1,AMZN,1871.72,1855.32,1897.92,1844.01,4713311,medium
2019-08-01,8,2019,1,AAPL,213.90,208.43,218.03,206.74,54017922,medium


## Creating a MultiIndex

In the previous lecture, we used the `set_index()` method to change the index from the default to something more meaningful. We can take this a set further and use more than one field as the index for our dataframe.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html

Let's start simply: instead of "date", we'll pass in a list of strings on which to set indices. 

In [43]:
tech.set_index(['date','name'])

month  year  day  ...      low    volume  volume_type
date       name                     ...                                
2014-01-02 FB         1  2014    2  ...    54.19  43257622       medium
           AAPL       1  2014    2  ...    78.86   8398851          low
           GOOGL      1  2014    2  ...   554.13   1822719       medium
           MSFT       1  2014    2  ...    37.10  30643745       medium
           AMZN       1  2014    2  ...   394.02   2140246       medium
...                 ...   ...  ...  ...      ...       ...          ...
2019-08-23 MSFT       8  2019   23  ...   132.80  38515386       medium
           AAPL       8  2019   23  ...   201.00  46882843       medium
           GOOGL      8  2019   23  ...  1150.00   1813141       medium
           AMZN       8  2019   23  ...  1745.23   5277898       medium
           FB         8  2019   23  ...   176.66  17331221       medium

[7105 rows x 9 columns]

The result is a MultiIndex, in which a single index has more than one component to it. When we promoted the "date" and "name" columns to indices, they were removed as regular columns from the dataframe. 

This looks a bit weird initially. But we'll get used to it.

Let's first set the `inplace` parameter to `True`.

In [44]:
tech.set_index(['date','name'], inplace = True)

Now let's look at the first two days.

In [45]:
tech.head(10)

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2014-01-02 FB         1  2014    2  ...   54.19  43257622       medium
           AAPL       1  2014    2  ...   78.86   8398851          low
           GOOGL      1  2014    2  ...  554.13   1822719       medium
           MSFT       1  2014    2  ...   37.10  30643745       medium
           AMZN       1  2014    2  ...  394.02   2140246       medium
2014-01-03 FB         1  2014    3  ...   54.53  38287706       medium
           GOOGL      1  2014    3  ...  552.47   1669229       medium
           MSFT       1  2014    3  ...   36.60  31134795       medium
           AAPL       1  2014    3  ...   77.20  14043410          low
           AMZN       1  2014    3  ...  396.22   2213512       medium

[10 rows x 9 columns]

In our dataframe, we see that there are two bold-faced columns: the date and the name of the stock. The date is no longer repeating, where blank spaces indicate the data from above is carried on. 

Hierarchical indexing is not only about looks, but by using a multiindex, we're creating a hierarchy of relationships within our data where the information across the two index levels is inseparable from the actual values.

Let's see what type of object this is.


In [46]:
type(tech.index)

pandas.core.indexes.multi.MultiIndex

We see that our index is now of the type "MultiIndex". Cool!

## MultiIndex from `read_csv()`

It turns out that we don't have to wait until the dataframe is read in to set a multiindex. Instead, we can set the multiindex (or even a single index for that matter) when reading in the dataframe with `read_csv`!

All we really need to do is pass in a list ofthe index columns to the `index_col` parameter.

In [47]:
pd.read_csv(tech_url, index_col = ['date','name'])

month  year  day  ...      low    volume  volume_type
date       name                     ...                                
2014-01-02 FB         1  2014    2  ...    54.19  43257622       medium
           AAPL       1  2014    2  ...    78.86   8398851          low
           GOOGL      1  2014    2  ...   554.13   1822719       medium
           MSFT       1  2014    2  ...    37.10  30643745       medium
           AMZN       1  2014    2  ...   394.02   2140246       medium
...                 ...   ...  ...  ...      ...       ...          ...
2019-08-23 MSFT       8  2019   23  ...   132.80  38515386       medium
           AAPL       8  2019   23  ...   201.00  46882843       medium
           GOOGL      8  2019   23  ...  1150.00   1813141       medium
           AMZN       8  2019   23  ...  1745.23   5277898       medium
           FB         8  2019   23  ...   176.66  17331221       medium

[7105 rows x 9 columns]

There are other ways create multiindex dataframes as well, but we won't cover them here. Generally speaking, those other methods should only be utilized for very specific cases that necessitate them.



## Indexing Hierarchical DataFrames

So we've set hierarchical dataframes. Now how do we extract values from them? When we set the multiindex, we created an association between each pair of date and stock ticker and a value in our dataframe. Thus, indexing the data will look a bit different.

In [48]:
tech.head(10)

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2014-01-02 FB         1  2014    2  ...   54.19  43257622       medium
           AAPL       1  2014    2  ...   78.86   8398851          low
           GOOGL      1  2014    2  ...  554.13   1822719       medium
           MSFT       1  2014    2  ...   37.10  30643745       medium
           AMZN       1  2014    2  ...  394.02   2140246       medium
2014-01-03 FB         1  2014    3  ...   54.53  38287706       medium
           GOOGL      1  2014    3  ...  552.47   1669229       medium
           MSFT       1  2014    3  ...   36.60  31134795       medium
           AAPL       1  2014    3  ...   77.20  14043410          low
           AMZN       1  2014    3  ...  396.22   2213512       medium

[10 rows x 9 columns]

Suppose we want to know what price Google closed at on January 2, 2014. Let's first try selecting for that particular date.

In [49]:
tech.loc['2014-01-02']

,month,year,day,open,close,high,low,volume,volume_type
name,,,,,,,,,
FB,1,2014,2,54.86,54.71,55.22,54.19,43257622,medium
AAPL,1,2014,2,79.38,79.02,79.58,78.86,8398851,low
GOOGL,1,2014,2,557.73,556.56,558.88,554.13,1822719,medium
MSFT,1,2014,2,37.35,37.16,37.40,37.10,30643745,medium
AMZN,1,2014,2,398.80,397.97,399.36,394.02,2140246,medium


Now we've isolated the full dataset for that date. To avoid being redundant, Pandas automatically drops the index that we are indexing along (in this case, "date").

Next, let's extract the Google data by going back to our `loc[]` indexer and adding another label to it.

In [60]:
tech.loc['2014-01-02', 'GOOGL']

month                1
year              2014
day                  2
open            557.73
close           556.56
high            558.88
low             554.13
volume         1822719
volume_type     medium
Name: (2014-01-02, GOOGL), dtype: object

What returns is a Series containing the values for Google on that single date. Now to isolate to close price, we can simply grab it out of the series as an attribute.

In [61]:
tech.loc['2014-01-02', 'GOOGL'].close

556.56

Another perhaps more elegant way of doing the same thing is to take advantage of the "type coupling" that we have between the "date" and "name" fields. They are both different levels in our multiindex, and so we can treat that as one dimension and capture it in a single tuple of values. In other words, we can identify the rows by the multiindex in one go by using a tuple, instead of performing this task iteratively.  By using a tuple, we are able to pass in multiple index labels while remaining in the first dimension (rows) of the `loc[]` indexer.
* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html

In [64]:
tech.loc[('2014-01-02', 'GOOGL')]

month                1
year              2014
day                  2
open            557.73
close           556.56
high            558.88
low             554.13
volume         1822719
volume_type     medium
Name: (2014-01-02, GOOGL), dtype: object

Best of all, we can go in and make use of the second dimension (columns) within `loc[]`.

In [65]:
tech.loc[('2014-01-02', 'GOOGL'), 'close']

556.56

What about selecting by position using `iloc[]`? This actually works the exact same way in multiindex dataframs as it does in single index dataframes. The hierarchical structure has no impact on how `iloc[]` functions.

In [69]:
tech.head()

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2014-01-02 FB         1  2014    2  ...   54.19  43257622       medium
           AAPL       1  2014    2  ...   78.86   8398851          low
           GOOGL      1  2014    2  ...  554.13   1822719       medium
           MSFT       1  2014    2  ...   37.10  30643745       medium
           AMZN       1  2014    2  ...  394.02   2140246       medium

[5 rows x 9 columns]

For example, on January 2, 2014 Google is at index position 2 (index starting at 0) in the dataframe, with the closing price as column position 4 (again index starting at 0). To get that value, we just use `iloc[]` the way we always have.

In [70]:
tech.iloc[2,4]

556.56

As another example, let's select the opening and closing prices for Apple on January 3, 2021.

In [71]:
tech.head(10)

month  year  day  ...     low    volume  volume_type
date       name                     ...                               
2014-01-02 FB         1  2014    2  ...   54.19  43257622       medium
           AAPL       1  2014    2  ...   78.86   8398851          low
           GOOGL      1  2014    2  ...  554.13   1822719       medium
           MSFT       1  2014    2  ...   37.10  30643745       medium
           AMZN       1  2014    2  ...  394.02   2140246       medium
2014-01-03 FB         1  2014    3  ...   54.53  38287706       medium
           GOOGL      1  2014    3  ...  552.47   1669229       medium
           MSFT       1  2014    3  ...   36.60  31134795       medium
           AAPL       1  2014    3  ...   77.20  14043410          low
           AMZN       1  2014    3  ...  396.22   2213512       medium

[10 rows x 9 columns]

The label-based approach with `loc[]` would be as follows:

In [75]:
tech.loc[('2014-01-03', 'AAPL'), ['open', 'close']]

open        79
close    77.28
Name: (2014-01-03, AAPL), dtype: object

The position-based approach with `iloc[]` would be:

In [74]:
tech.iloc[8, [3,4]]

open        79
close    77.28
Name: (2014-01-03, AAPL), dtype: object

The result is exactly the same!